<a href="https://colab.research.google.com/github/vohoaidanh/AIGCDetectBenchmark/blob/main/colab/DIRE_preprocess_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ZhendongWang6/DIRE.git

Cloning into 'DIRE'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 63 (delta 7), reused 59 (delta 6), pack-reused 0
Receiving objects: 100% (63/63), 1.95 MiB | 10.99 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!wget https://openaipublic.blob.core.windows.net/diffusion/jul-2021/256x256_diffusion_uncond.pt

--2024-03-25 13:42:58--  https://openaipublic.blob.core.windows.net/diffusion/jul-2021/256x256_diffusion_uncond.pt
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 20.209.18.33
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|20.209.18.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2211383297 (2.1G) [application/octet-stream]
Saving to: ‘256x256_diffusion_uncond.pt’

256x256_diffusion_u 100%[===================>]   2.06G  53.0MB/s    in 38s     

2024-03-25 13:43:36 (56.0 MB/s) - ‘256x256_diffusion_uncond.pt’ saved [2211383297/2211383297]



In [4]:
import os
os.makedirs('DIRE/models', exist_ok=True)
import shutil
shutil.move('/content/256x256_diffusion_uncond.pt', 'DIRE/models')


'DIRE/models/256x256_diffusion_uncond.pt'

In [5]:
!pip install ftfy -q
!pip install natsort -q
!pip install tensorboardX -q
!pip install blobfile -q
!pip install mpi4py


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 858.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.5-cp310-cp310-linux_x86_64.whl size=2746508 sha256=7d38084fb1fd64d2ea9009bfc0146662bfe31837c623fa70b77001837f197648
  Stored in directory: /root/.cache/pip/wheels/18/2b/7f/c852523089e9182b45fca50ff56f49a51eeb6284fd25a66713
Successfully built mpi4py


In [6]:
!cp /content/drive/MyDrive/DATASETS/real_gen_dataset.zip -d /content

In [7]:
!unzip -q /content/real_gen_dataset.zip -d DIRE

In [8]:
%cd DIRE

/content/DIRE


#Process for test set
(We need to remove all lines related to multi-GPU configuration in compe_dire.py before run)

In [ ]:
!python guided-diffusion/compute_dire.py \
--model_path models/256x256_diffusion_uncond.pt \
--attention_resolutions 32,16,8 \
--class_cond False \
--diffusion_steps 1000 --dropout 0.1 --image_size 256 --learn_sigma True --noise_schedule linear --num_channels 256 \
--num_head_channels 64 --num_res_blocks 2 --resblock_updown True --use_fp16 True --use_scale_shift_norm True \
--images_dir /content/DIRE/real_gen_dataset/test \
--recons_dir /content/drive/MyDrive/DATASETS/DIRE/DiffusionForensics/recons_test \
--dire_dir /content/drive/MyDrive/DATASETS/DIRE/DiffusionForensics/dire_test \
--batch_size 16 --num_samples 1000  --timestep_respacing ddim20 --use_ddim True \
--has_subfolder True

#Process for train set

In [9]:
import os
image_list = []
extensions = ['.jpg', '.png', '.gif', '.bmp', '.webp']  # Add more extensions as needed
def getname(base_dir):
    for dirpath, dirnames, filenames in os.walk(base_dir):
        if len(filenames) > 0:
            imgs = [filename for filename in filenames if any(ext in filename for ext in extensions)]
            image_list.extend(imgs)
    return image_list


In [10]:
image_list = getname('/content/drive/MyDrive/DATASETS/DIRE/DiffusionForensics/dire_train')

In [11]:
len(image_list)

1896

In [ ]:

#Remove existed items before run
data_dir = '/content/DIRE/real_gen_dataset'
c=0
for dirpath, dirnames, filenames in os.walk(data_dir):
    if len(filenames) >0:
        for file in filenames:
            if file in image_list:
                c+=1
                full_name = os.path.join(dirpath, file)
                os.remove(full_name)
                print('removed {}:{}:'.format(c,full_name))


In [13]:
len(os.listdir('/content/DIRE/real_gen_dataset/train/0_real')) + len(os.listdir('/content/DIRE/real_gen_dataset/train/1_fake'))

1686

In [16]:
samples = len(os.listdir('/content/DIRE/real_gen_dataset/train/0_real')) + len(os.listdir('/content/DIRE/real_gen_dataset/train/1_fake'))

In [18]:
!python guided-diffusion/compute_dire.py \
--model_path models/256x256_diffusion_uncond.pt \
--attention_resolutions 32,16,8 \
--class_cond False \
--diffusion_steps 1000 --dropout 0.1 --image_size 256 --learn_sigma True --noise_schedule linear --num_channels 256 \
--num_head_channels 64 --num_res_blocks 2 --resblock_updown True --use_fp16 True --use_scale_shift_norm True \
--images_dir /content/DIRE/real_gen_dataset/train \
--recons_dir /content/drive/MyDrive/DATASETS/DIRE/DiffusionForensics/recons_train \
--dire_dir /content/drive/MyDrive/DATASETS/DIRE/DiffusionForensics/dire_train \
--batch_size 16 --num_samples $samples  --timestep_respacing ddim20 --use_ddim True \
--has_subfolder True

Logging to /content/drive/MyDrive/DATASETS/DIRE/DiffusionForensics/recons_train
Namespace(images_dir='/content/DIRE/real_gen_dataset/train', recons_dir='/content/drive/MyDrive/DATASETS/DIRE/DiffusionForensics/recons_train', dire_dir='/content/drive/MyDrive/DATASETS/DIRE/DiffusionForensics/dire_train', clip_denoised=True, num_samples=1686, batch_size=16, use_ddim=True, model_path='models/256x256_diffusion_uncond.pt', real_step=0, continue_reverse=False, has_subfolder=True, image_size=256, num_channels=256, num_res_blocks=2, num_heads=4, num_heads_upsample=-1, num_head_channels=64, attention_resolutions='32,16,8', channel_mult='', dropout=0.1, class_cond=False, use_checkpoint=False, use_scale_shift_norm=True, resblock_updown=True, use_fp16=True, use_new_attention_order=False, learn_sigma=True, diffusion_steps=1000, noise_schedule='linear', timestep_respacing='ddim20', use_kl=False, predict_xstart=False, rescale_timesteps=False, rescale_learned_sigmas=False)
have created model and diffusi